### 라이브러리 불러오기

In [13]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2
import gc

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time
import GPUtil


device = torch.device('cuda')

In [2]:
train_png = sorted(glob('train/*.png'))
test_png = sorted(glob('test/*.png'))

In [3]:
train_y = pd.read_csv("open/train_df.csv")

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

In [4]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (224, 224))
    return img

In [5]:
train_imgs = [img_load(m) for m in tqdm(train_png)]
test_imgs = [img_load(n) for n in tqdm(test_png)]

100%|██████████| 2154/2154 [01:01<00:00, 35.30it/s]


In [6]:
meanRGB = [np.mean(x, axis=(0,1)) for x in train_imgs]
stdRGB = [np.std(x, axis=(0,1)) for x in train_imgs]

meanR = np.mean([m[0] for m in meanRGB])/255
meanG = np.mean([m[1] for m in meanRGB])/255
meanB = np.mean([m[2] for m in meanRGB])/255

stdR = np.mean([s[0] for s in stdRGB])/255
stdG = np.mean([s[1] for s in stdRGB])/255
stdB = np.mean([s[2] for s in stdRGB])/255

print("train 평균: {:.6f}, {:.6f}, {:.6f}".format(meanR, meanG, meanB))
print("train 표준편차: {:.6f}, {:.6f}, {:.6f}".format(stdR, stdG, stdB))

train 평균 0.4326520637590727 0.40305811291301064 0.39374737462935844
train 표준편차 0.18287215637945708 0.17509091131647034 0.16429985101228456


In [7]:
meanRGB = [np.mean(x, axis=(0,1)) for x in test_imgs]
stdRGB = [np.std(x, axis=(0,1)) for x in test_imgs]

meanR = np.mean([m[0] for m in meanRGB])/255
meanG = np.mean([m[1] for m in meanRGB])/255
meanB = np.mean([m[2] for m in meanRGB])/255

stdR = np.mean([s[0] for s in stdRGB])/255
stdG = np.mean([s[1] for s in stdRGB])/255
stdB = np.mean([s[2] for s in stdRGB])/255

print("test 평균: {:.6f}, {:.6f}, {:.6f}".format(meanR, meanG, meanB))
print("test 표준편차: {:.6f}, {:.6f}, {:.6f}".format(stdR, stdG, stdB))

test 평균 0.4178757104484087 0.3927157140929144 0.3862456383580592
test 표준편차 0.196168942855468 0.19093924508013493 0.1814009497154592


In [19]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode == 'train':
            train_transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.RandomAffine((-45, 45)),
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.432652, 0.403058, 0.393747],
                                     std = [0.182872, 0.175091, 0.164300])
            ])
            img = train_transform(img)
        if self.mode == 'test':
            test_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.417876, 0.392716, 0.386246],
                                     std = [0.196169, 0.190939, 0.181400])
            ])
            img = test_transform(img)
        
        label = self.labels[idx]
        return img, label

In [83]:
batch_size = 32
epochs = 30

# Train
train_dataset = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train')
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

In [76]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = timm.create_model('efficientnet_b3', pretrained=True, num_classes=88)
        
    def forward(self, x):
        x = self.model(x)
        return x

### 모델 학습

In [84]:
gc.collect()
torch.cuda.empty_cache()

In [85]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

model = Network().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler() 

best=0
train_losses = []
for epoch in range(epochs):
    GPUtil.showUtilization()
    start=time.time()
    train_loss = 0
    running_corrects = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
        
    train_f1 = score_function(train_y, train_pred)
    
    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
    train_losses.append(train_loss)

| ID | GPU | MEM |
------------------
|  0 | 13% | 47% |
epoch : 1/30    time : 128s/3721s
TRAIN    loss : 1.05752    f1 : 0.22126
| ID | GPU | MEM |
------------------
|  0 | 94% | 84% |
epoch : 2/30    time : 129s/3613s
TRAIN    loss : 0.55431    f1 : 0.39004
| ID | GPU | MEM |
------------------
|  0 | 95% | 84% |
epoch : 3/30    time : 130s/3499s
TRAIN    loss : 0.45726    f1 : 0.47394
| ID | GPU | MEM |
------------------
|  0 | 95% | 85% |
epoch : 4/30    time : 129s/3354s
TRAIN    loss : 0.36739    f1 : 0.55101
| ID | GPU | MEM |
------------------
|  0 | 93% | 85% |
epoch : 5/30    time : 128s/3200s
TRAIN    loss : 0.30265    f1 : 0.62591
| ID | GPU | MEM |
------------------
|  0 | 91% | 85% |
epoch : 6/30    time : 120s/2885s
TRAIN    loss : 0.26659    f1 : 0.66495
| ID | GPU | MEM |
------------------
|  0 | 91% | 85% |
epoch : 7/30    time : 119s/2738s
TRAIN    loss : 0.20879    f1 : 0.74254
| ID | GPU | MEM |
------------------
|  0 | 94% | 84% |
epoch : 8/30    time : 118

In [86]:
torch.save(model, f'eNetb3_bs=32_epoch=30.pt')

### 추론

In [87]:
model.eval()
f_pred = []

with torch.no_grad():
    for batch in (test_loader):
        x = torch.tensor(batch[0], dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

In [88]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

### 제출물 생성

In [89]:
submission = pd.read_csv("open/sample_submission.csv")

submission["label"] = f_result

submission

,index,label
0,0,tile-glue_strip
1,1,grid-good
2,2,transistor-good
3,3,tile-gray_stroke
4,4,tile-good
...,...,...
2149,2149,tile-gray_stroke
2150,2150,screw-good
2151,2151,grid-good
2152,2152,cable-good


In [90]:
submission.to_csv("eNetb3_bs=32_epoch=30.csv", index = False)

In [114]:
import pickle

with open('train_losses_eNet.p', 'wb') as f:
    pickle.dump(train_losses, f)

In [92]:
train_losses

[1.0575188024720152,
 0.5543067135027985,
 0.45725534923041056,
 0.36739429075326496,
 0.3026542094216419,
 0.2665929082614273,
 0.2087919320633163,
 0.19555407851489634,
 0.19193005917677247,
 0.1588310412506559,
 0.14932444558214797,
 0.11712785977036201,
 0.13821738513547982,
 0.12667590468677123,
 0.12256250808488071,
 0.09995594309337107,
 0.10577917810696277,
 0.12948770665410742,
 0.16090119774661846,
 0.11790420759969684,
 0.09252452850341793,
 0.07804243600190572,
 0.07772009764144673,
 0.04827625004213247,
 0.0647156416480221,
 0.06755750570724263,
 0.06660430822799455,
 0.10735380827490962,
 0.09603406422173798,
 0.06929409681861082]

In [97]:
f_pred

[62,
 28,
 72,
 64,
 63,
 50,
 17,
 32,
 40,
 39,
 0,
 76,
 85,
 56,
 61,
 2,
 25,
 20,
 21,
 47,
 66,
 87,
 0,
 58,
 54,
 55,
 52,
 60,
 40,
 78,
 55,
 9,
 55,
 84,
 84,
 82,
 40,
 9,
 37,
 44,
 45,
 6,
 58,
 14,
 44,
 63,
 68,
 15,
 84,
 55,
 9,
 72,
 31,
 35,
 84,
 46,
 20,
 21,
 38,
 15,
 73,
 50,
 18,
 85,
 17,
 36,
 72,
 13,
 53,
 43,
 25,
 76,
 32,
 63,
 41,
 84,
 9,
 45,
 9,
 9,
 50,
 76,
 57,
 77,
 72,
 0,
 84,
 21,
 45,
 83,
 36,
 84,
 9,
 42,
 55,
 65,
 72,
 72,
 45,
 45,
 9,
 21,
 52,
 15,
 76,
 6,
 85,
 49,
 72,
 34,
 36,
 58,
 72,
 70,
 84,
 15,
 28,
 52,
 15,
 41,
 55,
 19,
 45,
 73,
 77,
 72,
 61,
 72,
 49,
 55,
 28,
 46,
 50,
 42,
 15,
 25,
 55,
 2,
 52,
 78,
 72,
 33,
 7,
 9,
 84,
 35,
 23,
 84,
 85,
 76,
 84,
 76,
 73,
 52,
 84,
 63,
 68,
 63,
 33,
 45,
 57,
 60,
 55,
 6,
 21,
 38,
 49,
 74,
 83,
 10,
 21,
 24,
 55,
 39,
 9,
 52,
 55,
 86,
 15,
 84,
 21,
 25,
 3,
 6,
 52,
 20,
 62,
 45,
 79,
 73,
 52,
 18,
 84,
 32,
 52,
 55,
 83,
 85,
 84,
 33,
 72,
 9,
 58,
 1,
 52